# Minimising Churn Rate Of Subscription Products

Subscription products often are the main source of revenue for companies across all industries, such as Netflix. These products may come in the form of a 'one size fits all' overcompassing subscription, or in multi-level memberships. Regardless of how they structure their memberships, or what industry they are in, companies usually try to minimise customer churn (a.k.a. subscription cancellations). To retain their customers, these companies are required to firstly identify behavioural patterns that act as a catalyst in disengagement with the product.

- **Market:** The target audience is the entire company's subscription base. They are the ones the company wants to keep.

- **Product:** The subscription products that customers are already enrolled in can provide value that users may not have imagined, or they may have forgotten.

- **Goal:** The objective of this model is to predict which users are likely to churn, so that the company can focus on re-engaging these users with the product. These efforts can be notifications about the benefits of the product, especially focusing on features that are new or that the user has shown to value.
<br><br>
- In this case study we will be working for a fintech that provides a subscription product to its users, which allows them to manage their bank accounts (saving accounts, credit cards ... etc), provides them with personalised coupons, informs them of the latest low-APR loans available in the market, and educates them on the best available methods to save money (e.g. videos or saving money on taxes, free courses on financial health ... etc).
- We are in charge of identifying users who are likely to cancel their subscription so that we can start building new features that they may be interested in. These features can increase the engagement and interest of our users towards the product.
<br><br>
- By subscribing to the membership, our customers have provided us with the data on their finances, as well as how they handle those finances through the product. We also have some demographic information we acquired from them during the sign-up process.
- Financial data can often be unreliable and delayed. As a result, companies can build their marketing models using demographic data, and data related to finances handled through the product itself. Therefore, we will be restricting ourselves to only using that type of data. Furthermore, product-related data is more indicative of what new features we should be creating as a company.

# Importing The Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random

# Importing The Dataset

In [ ]:
ds = pd.read_csv('churn_data.csv')

In [ ]:
ds.head()

In [ ]:
ds.describe()

# Taking Care Of Missing Data

In [ ]:
ds.isna().any()

In [ ]:
# This can be displayed graphically

plt.rcParams['figure.figsize'] = (16,4)
sns.heatmap(ds.isnull(), yticklabels = False, cbar = False, cmap = 'Blues')

In [ ]:
# From the visualisation above, we do not see any missing values for age. 
# We know there are missing values for age as the code above states 'True'for age. 
# From the code below, we observe only 4 missing values for age.
# The above visualisation now makes sense as there is a large number of rows
# (27,000) and as there are only 4 missing values, the line will be too thin to see.

ds.isna().sum()

In [ ]:
# Instead of setting the blank columns of age to the average age, we will simply remove these rows.
# The code below will return the dataset except any rows where the 'age' column had any blanks.

ds = ds[pd.notnull(ds['age'])]

In [ ]:
# We observe 26,996 rows as opposed to 27,000 rows, this being a result of removing the rows where 
# the age column was blank.

ds.shape

In [ ]:
# As the columns 'credit_score' and 'rewards_earned' contain a lot of blanks, we will remove these columns.
# Note: These columns have been removed from our model and analysis.

ds = ds.drop(columns = ['credit_score', 'rewards_earned'])

In [ ]:
# We observe 29 columns, 2 less from the original 31.

ds.shape

In [ ]:
ds.head()

# Visualising the dataset

In [ ]:
ds.info()

In [ ]:
ds.head()

In [ ]:
ds[ds.housing == 'O'].churn.value_counts()

In [ ]:
ds[ds.churn == 0].housing.value_counts()

In [ ]:
ds2 = ds.drop(columns = ['user'])

# Histograms

In [ ]:
column_names = ds2.columns

In [ ]:
column_names[3]

In [ ]:
plt.suptitle('Histograms of Columns', fontsize = 20)
for i in column_names:
    plt.title(i)
    plt.hist(ds2[i])
    plt.show()

# Pie Charts

**Note:** We will be making Pie Charts for the binary variables. This is because, if the binary variables are disproportionate (i.e. 99% are 0 and only 1% is 1), and the response variable for the disproportionate binary variable has a majority 0 or 1 as a response. Regardless of the other variables, that particular binary variable will have a greater influence on the response variable, resulting in the predicted response being highly dependent/correlated on that binary variable. We do not wish there to be a higher than desired dependence on any variable.

In [ ]:
# Pie Chart for the Churn

plt.suptitle('Churned Pie Chart', fontsize = 20)
plt.pie([ds[ds['churn'] == 0]['churn'].count(), ds[ds['churn'] == 1]['churn'].count()],
            labels = ['0','1'], colors = ['r','g'], startangle = 90, autopct='%.1f%%')
plt.show()

In [ ]:
# We wish to only visualise Pie Charts for the binary variables.

ds2 = ds[['is_referred', 'app_downloaded',
          'web_user', 'app_web_user', 'ios_user',
          'android_user','waiting_4_loan', 'cancelled_loan',
          'received_loan', 'rejected_loan', 
          'left_for_two_month_plus', 'left_for_one_month']]

In [ ]:
columns = ds2.columns

In [ ]:
ds2.head()

In [ ]:
ds2[ds2['is_referred'] == 0]['is_referred'].count()

In [ ]:
ds2[ds2['is_referred'] == 1]['is_referred'].count()

In [ ]:
# We observe the following columns being highly disproportionate: 
# app_downloaded, waiting_4_loan, cancelled_loan, 
# received_loan, rejected_loan, left_for_one_month. 

plt.suptitle('Pie Chart Distribution', fontsize = 20)
for i in columns:
    plt.title(i)
    plt.pie([ds2[ds2[i] == 0][i].count(), ds2[ds2[i] == 1][i].count()],
            labels = ['0','1'], colors = ['r','g'], startangle = 90, autopct='%.1f%%')
    plt.show()

In [ ]:
# We will be observing the value counts of the above mentioned columns to check if either
# binary value of 0 or 1 has a very high/low proportion of the churn. If it does, we may
# decide to skip that column due to high correlation or something being suspicious.

print(ds[ds.app_downloaded == 0].churn.value_counts())
print(ds[ds.app_downloaded == 1].churn.value_counts())

In [ ]:
print(ds[ds.waiting_4_loan == 0].churn.value_counts())
print(ds[ds.waiting_4_loan == 1].churn.value_counts())

In [ ]:
print(ds[ds.cancelled_loan == 0].churn.value_counts())
print(ds[ds.cancelled_loan == 1].churn.value_counts())

In [ ]:
print(ds[ds.received_loan == 0].churn.value_counts())
print(ds[ds.received_loan == 1].churn.value_counts())

In [ ]:
print(ds[ds.rejected_loan == 0].churn.value_counts())
print(ds[ds.rejected_loan == 1].churn.value_counts())

In [ ]:
print(ds[ds.left_for_one_month == 0].churn.value_counts())
print(ds[ds.left_for_one_month == 1].churn.value_counts())

**Summary:** We have not observed any column that has a high bias/disproportion of 0's or 1's per any binary. We therefore conclude there is no strong bias in the dataset and we can move onto the next steps.

# Correlation Plot

In [ ]:
# Example observations/interpretations

# The younger you are, the more likely you are to churn.
# The less deposits, withdrawals, purchases partners and purchases the more likely to churn.
# The more cc_taken (credit cards taken), the more likely to churn. Perhaps users dislike the credit card services.
# The more recommendations we give to users, the less likely they are to churn.


ds.drop(columns = ['churn', 'user','housing','payment_type','zodiac_sign']).corrwith(ds.churn).plot.bar(
        figsize = (20,10), title = 'Correlation with Churn', fontsize = 15, rot = 90, grid = True)

In [ ]:
# Correlation matrix of independent variables

sns.set(style = 'white', font_scale = 1.1) # Builds the background

# Compute the correlation matrix
corr = ds.drop(columns = ['user','churn']).corr() # Creating a 2D array of each correlation feature to each other

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype = np.bool)
mask[np.triu_indices_from(mask)] = True # This creates a the lower diagonal of the matrix as it is symmetrical

# Set up the matplotlib figure
fig, axes = plt.subplots(figsize = (9,9)) # Size of the plot
fig.suptitle("Correlation Matrix", fontsize = 40) # Title

# Generate a custom diverging colourmap

cmap = sns.diverging_palette(220, 10, as_cmap = True) # Colouring

# Draw the heatmap with the mask and correct aspect ratio

sns.heatmap(corr, mask = mask, cmap = cmap, vmax = 0.4, center = 0, 
            square = True, linewidth = 0.5, cbar_kws = {'shrink': 0.5})

# We observe a strong negative correlation between Android and iOS users which is understandable
# as, if you are an Android user you will not use iOS and vice versa.

In [ ]:
# From a senior analyst, we have received a useful tip that the app_web_user variable
# is actually a function of web_user and app_downloaded. Therefore these variables are
# no longer independent. We will therefore remove the app_web_user column from the dataset
# and save it as a new csv file.

ds = ds.drop(columns = 'app_web_user')
ds = ds.to_csv('new_churn_data.csv', index = False)

**Note:** One of the main reasons why we wish to build a model with the least or optimal amount of variables is that, as a data analyst we should understand the data well. If we have many variables, we will be expected to know the significance of each variable and its ability to impact other variables and the responding variable. Although the correlation plot and the correlation matrix may indicate this, it does not explicitly tell us these in depth details.  

In [ ]:
ds = pd.read_csv('new_churn_data.csv')

In [ ]:
ds.head()

In [ ]:
user_identifier = ds.user
ds = ds.drop(columns = 'user')

# Taking care of categorical data

In [ ]:
ds.housing.value_counts()

In [ ]:
ds = pd.get_dummies(ds)

In [ ]:
ds.columns

In [ ]:
ds.head()

In [ ]:
ds.shape

# Avoiding the dummy variable trap

In [ ]:
ds = ds.drop(columns = ['housing_na', 'zodiac_sign_na', 'payment_type_na'])

In [ ]:
ds.shape

# Splitting the dataset into the training set and test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(ds.drop(columns = 'churn'), ds.churn, test_size = 0.2, random_state = 0)

In [ ]:
X_train

In [ ]:
y_train

# Balancing the dataset

From the below we observe that the training dataset response variable contains around 60% 0's and 40% 1's. If we are building a model and the model predicts all values to be 0, the accuracy will be deemed 60% but it will be misleading. We will therefore need to balance the 0's and 1's in the response variable. If we have 50% 0's and 50% 1's, any additional accuracy the model produces will be seen as a result of the model being good, as opposed to a permutation of the results from the response variable.

In [ ]:
y_train.value_counts()

In [ ]:
pos_index = y_train[y_train == 1].index
neg_index = y_train[y_train == 0].index

In [ ]:
# The below sets which is the higher list and which is the lower list.

if len(pos_index) > len(neg_index):
    higher = pos_index
    lower = neg_index
else:
    lower = pos_index
    higher = neg_index

In [ ]:
higher.shape

In [ ]:
lower.shape

In [ ]:
random.seed(0)

In [ ]:
# This makes the higher and lower list the same size by randomly selecting the 
# same number of items from the higher list as there are in the lower list.

higher = np.random.choice(higher, size = len(lower))

In [ ]:
type(higher)

In [ ]:
type(lower)

In [ ]:
# To make higher and lower list the same type for consistency.

lower = np.asarray(lower)

In [ ]:
new_indexes = np.concatenate((lower,higher))

In [ ]:
new_indexes.shape

In [ ]:
lower.shape

In [ ]:
higher.shape

In [ ]:
X_train = X_train.loc[new_indexes, ]
y_train = y_train.loc[new_indexes]

In [ ]:
X_train

In [ ]:
y_train

# Feature Scaling

**Note:** The Standard Scaler returns a numpy array of multiple dimensions. The problem with this process is that it loses the column names and index. The index is how we identify each set of fields to the user, and we would like the column names to be build within our model. We therefore save the scaled part into a different data frame by converting the result of the Standard Scaler into its data frame.

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()

In [ ]:
X_train2 = pd.DataFrame(sc_X.fit_transform(X_train))
X_test2 = pd.DataFrame(sc_X.transform(X_test))

In [ ]:
X_train2.columns = X_train.columns.values
X_test2.columns = X_test.columns.values

X_train2.index = X_train.index.values
X_test2.index = X_test.index.values

In [ ]:
X_train = X_train2
X_test = X_test2

In [ ]:
X_train

In [ ]:
X_test

# Fitting the Logistic Regression to the dataset.

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state = 0)

In [ ]:
lr.fit(X_train, y_train)

In [ ]:
# Predicting the test set results

y_pred = lr.predict(X_test)

# Model Evaluation - Confusion Matrix and K-Fold Cross Validation

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, precision_score, recall_score
cm = confusion_matrix(y_test, y_pred)
print(cm)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
# This is a function of precision_score and recall_score, and it balances
# them out.
f1_score(y_test, y_pred)

In [ ]:
# (True Positives)/(True Positives + False Positives)
# Of all the positives predicted, how many of them are correct.

precision_score(y_test, y_pred)

In [ ]:
# (True Positives)/(True Positives + False Negatives)
# Of all the positives that truly exist, how many did we predict as true.

recall_score(y_test, y_pred)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import cross_val_score
cvs = cross_val_score(estimator = lr, X = X_train, y = y_train, cv = 10)
accuracy_mean = cvs.mean()
accuracy_std = cvs.std()
print(cvs)
print(accuracy_mean)
print(accuracy_std)

# Analysing Coefficients

In [ ]:
pd.concat([pd.DataFrame(X_train.columns, columns = ['Features']),
           pd.DataFrame(np.transpose(lr.coef_), columns = ['Coef'])],
           axis = 1)

# Feature Selection

With less features and the same accuracy, the model will be lighter and faster to run

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

In [ ]:
rfe = RFE(estimator = lr, n_features_to_select = 20)

In [ ]:
rfe = rfe.fit(X_train, y_train)

# Summarising the selection of the attributes

In [ ]:
print(rfe.support_)

In [ ]:
X_train.columns[rfe.support_]

In [ ]:
rfe.ranking_

# Fitting the Logistic Regression to the dataset

In [ ]:
lr.fit(X_train[X_train.columns[rfe.support_]], y_train)

In [ ]:
# Predicting the test set results

y_pred = lr.predict(X_test[X_test.columns[rfe.support_]])

# Model Evaluation - Confusion Matrix and K-Fold Cross Validation

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, precision_score, recall_score
cm = confusion_matrix(y_test, y_pred)
print(cm)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
# This is a function of precision_score and recall_score, and it balances
# them out.
f1_score(y_test, y_pred)

In [ ]:
# (True Positives)/(True Positives + False Positives)
# Of all the positives predicted, how many of them are correct.

precision_score(y_test, y_pred)

In [ ]:
# (True Positives)/(True Positives + False Negatives)
# Of all the positives that truly exist, how many did we predict as true.

recall_score(y_test, y_pred)

# Analysing Coefficients

In [ ]:
pd.concat([pd.DataFrame(X_train.columns[rfe.support_], columns = ['Features']),
           pd.DataFrame(np.transpose(lr.coef_), columns = ['Coef'])],
           axis = 1)

# Final Result

In [ ]:
results = pd.concat([y_test, user_identifier], axis = 1).dropna()
results['predicted_churn'] = y_pred
results = results[['user','churn','predicted_churn']].reset_index(drop = True)

In [ ]:
print(results)